In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
try:
    import torch
    import torchvision
    assert int(torch.__version__.split(".")[1]) >= 12, "torch version should be 1.12+"
    assert int(torchvision.__version__.split(".")[1]) >= 13, "torchvision version should be 0.13+"
    print(f"torch version1: {torch.__version__}")
    print(f"torchvision version1: {torchvision.__version__}")
except:
    print(f"[INFO] torch/torchvision versions not as required, installing nightly versions.")
    !pip3 install -U torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
    import torch
    import torchvision
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")
try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
import matplotlib.pyplot as plt
from torch import nn
from torchvision import transforms
from torchvision.models import  resnet18,ResNet18_Weights
from tqdm import tqdm
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
"""
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))"""

random_seed = 2023
np.random.seed(random_seed)
torch.manual_seed(random_seed)
random.seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = "cuda" if torch.cuda.is_available() else "cpu"
print("device ",device)
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
TRAIN_DIR = "/kaggle/input/44-cat-breed-dataset/cat dataset/train"
TEST_DIR ="/kaggle/input/44-cat-breed-dataset/cat dataset/test"



In [ ]:
data_dir = TRAIN_DIR
classes = os.listdir(data_dir)
classes=sorted(classes)
#classes.pop(-1)
count=0
for i in classes:
    print("no.items in class",i,"is",len(os.listdir(data_dir+"/"+i)))
    count+=len(os.listdir(data_dir+"/"+i))
    if i[0]=="M":
        print(i)
print(count)

In [ ]:
data=iter(dataset)
im,l=next(data)
show_image(im,l)

In [ ]:
def show_image(img, label):
    print('Label: ', dataset.classes[label], "("+str(label)+")")
    #cause matplotlib expects you to have the channels last insted of first like in torch
    plt.imshow(img.permute(1, 2, 0))

In [ ]:
#rom torch.datasets import ImageFolder
from torchvision.models import  resnet50,ResNet50_Weights
batch_size=64
train_transform =ResNet50_Weights.DEFAULT.transforms()

test_transform = transforms.Compose([
    transforms.Resize((250, 250)),
    transforms.ToTensor(), # ToTensor : [0, 255] -> [0, 1]
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_data = ImageFolder(TRAIN_DIR, train_transform)
test_data = ImageFolder(TEST_DIR, test_transform)
print("crying hi :)")
train_loader = DataLoader(train_data,
                          batch_size=batch_size,
                          shuffle=True)

test_loader = DataLoader(test_data, 
                         batch_size=batch_size,
                         shuffle=True)
image_datasets={"train":train_data,"test":test_data}
dataloaders={"train":train_loader,"test":test_loader}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}


In [ ]:
model_conv34 = resnet50(weights=ResNet50_Weights.DEFAULT).to(device)
"""for param in model_conv34.parameters():
    param.requires_grad = False
"""
num_ftrs = model_conv34.fc.in_features
model_conv34.fc = nn.Linear(num_ftrs, 42)

model_conv34 = model_conv34.to(device)

In [ ]:
import torch.optim as optim
from torch.optim import lr_scheduler
criterion = nn.CrossEntropyLoss()
optimizer_conv = torch.optim.SGD(model_conv34.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)


In [ ]:
import time
def train_modelv(model, criterion, optimizer, scheduler, phases,num_epochs=25 ):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in phases:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in tqdm(dataloaders[phase],desc=phase, position=0, leave=True):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
            if epoch % 5==0:
                torch.save(model,str(epoch)+str(best_acc)+"resnet34.pt")

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model


In [ ]:
import copy
model_conv34 = train_modelv(model_conv34, criterion, optimizer_conv,
                         exp_lr_scheduler,['train','test'], num_epochs=500)

I tried resnet34 without frezing and .0002 lr, for 56 and only got 30% tr_accuracy and 40% test_accur.
I tried resnet50 frozen and got 42% tr and 45% test for 32 epoch,lr=.001.
and started over with resnet50 with unfrozen layers. 9 epoch and I got 62% tr_accuracy and 56.5% test_accuracu, lr=.001, 16 epochs:64% and 57.16%, the loss is still decresing, thanks god.

epoch 18, incresing loss for test and decresing accuracy. 64% tr_accuracy, 56.5% test_accur, 
should I lower the lr????
it's epoch 22 and it keeps getting not so well, 23, decrese in tra_acc and test_acc
I stopped and will save the model and continue later with smaller lr.


In [ ]:
torch.save(model_conv34,"resnet50_30epochs_65%_tr_accuracy_57%_test_accuracy.pt")

In [ ]:
len(train_data.class_to_idx.keys()) #67 classes
train_data.class_to_idx